In [9]:
# The code was removed by Watson Studio for sharing.

In [10]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [8]:
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'xxx'
VERSION = '20180605' # Foursquare API version
LIMIT=100
radius=500

# Prep Work- Saving Data to File

The reason  I save the data we need to file is because my Four Square account has a quota on the number of calls I can make per day, so I will get all the data first ( to be used later from file), instead of calling Four Sqaure all the time during my anaysis

##  (1) Saving Manhattan/Brooklyn Location Data to file
###  Raw_NYC_BLY_Location.csv

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [4]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd

In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [34]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# Include only these neighborhoods in Brooklyn that we know has Vegetarian Restaurants
veg_neighborhood =[
'Greenpoint',
'Manhattan Terrace',
'Windsor Terrace',
'Prospect Heights',
'Williamsburg',
'Bushwick',
'Brooklyn Heights',
'Carroll Gardens',
'Fort Greene',
'Coney Island',
'Clinton Hill',
'Downtown',
'Boerum Hill',
'Prospect Lefferts Gardens',
'East Williamsburg',
'North Side',
'South Side',
'Fort Hamilton',
'Fulton Ferry',
'Vinegar Hill',
'Dumbo']



# Take only  Manhattan & Brooklyn Data
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name'] 
    if ((borough=="Manhattan") | 
        ((borough=="Brooklyn") & (neighborhood_name in veg_neighborhood))):   
      
       neighborhood_latlon = data['geometry']['coordinates']
       neighborhood_lat = neighborhood_latlon[1]
       neighborhood_lon = neighborhood_latlon[0]  
       neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
df_nyc = neighborhoods

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_nyc['Borough'].unique()),
        df_nyc.shape[0]
    )
)



The dataframe has 2 boroughs and 61 neighborhoods.


In [35]:
project.save_data(data= df_nyc.to_csv(index=False),file_name='Raw_NYC_BLY_Location.csv',overwrite=True)

{'file_name': 'Raw_NYC_BLY_Location.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'battleneighborhood-donotdelete-pr-daicfvssjqzcx5',
 'asset_id': 'd8520c97-1710-4b90-ac9a-00c52d60f2fe'}

In [12]:
def getTestData(df_nyc, neigh):
    row =  df_nyc[df_nyc["Neighborhood"] == neigh]
    lat =row.iloc[0]['Latitude']
    lng =row.iloc[0]['Longitude']
    return  neigh, lat, lng

In [13]:
def  getJsonField(thisObj, fieldname):
    if not fieldname in thisObj :
        return 'None'
    else:
        return thisObj[fieldname]

In [13]:
neigh, lat, lng = getTestData(df_nyc, "East Village")
print ("{0} {1},{2}".format(neigh, lat, lng))

East Village 40.727846777270244,-73.98222616506416


Function using Explore API for section=food to get popular food venue (max of 100) near each neighborhood

In [21]:
def getFoodVenues(names, latitudes, longitudes,  radius=500 ):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&section={}&intent=browse&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            "food",
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        print (name)
        results1 = requests.get(url).json()["response"]
        results2 = results1['groups']
        #print (results2)
        if ((results2 == None)  | (len(results2) == 0)):
            continue
        
        results3=results2[0]
        if (results3['items']   == None):
            continue
        results = results3['items']
        
        for thisR in results:
            v = thisR['venue']
            cat1 = getJsonField(v, 'categories')

            if (cat1 !='None'):
                #print ("cat " + str((cat1[0])))
                cat1= cat1[0]  # Take the first one from the list                
                cat = getJsonField(cat1, 'name')
                #print(cat)
            else:
               cat='None'
            venues_list.append([(
                 name, 
                 lat, 
                 lng, 
                 v['id'],
                 v['name'], 
                 v['location']['lat'], 
                 v['location']['lng'],  
                 cat)])
        
    

        cat_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        cat_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude',
                  'Id',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(cat_venues)

In [24]:
neigh, lat, lng = getTestData(df_nyc, "Park Slope")
food_venues = getFoodVenues([neigh], [lat], [lng], radius=500 )

Park Slope


In [40]:
all_food_venues = getFoodVenues(df_nyc['Neighborhood'],df_nyc['Latitude'],df_nyc['Longitude'],radius=500 )

In [37]:
project.save_data(data=all_food_venues.to_csv(index=False),file_name='Raw_NYC_BLY_Food_Venues.csv',overwrite=True)

{'file_name': 'Raw_NYC_BLY_Food_Venues.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'battleneighborhood-donotdelete-pr-daicfvssjqzcx5',
 'asset_id': '7ca5ee3a-4945-4b31-8fb7-5ab67b7bdcf1'}

# (3) Saving all Vegetarian Venues to file
### Raw_NYC_BLY_Veg_Venues.csv

Function using Search API for category=Vegetarian to get  venue (max of 100) near each neighborhood

In [49]:
def getVenuesOfCategory(names, latitudes, longitudes, category_query, radius=500 ):
    
    cat_venues=None
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
       
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&categoryId={}&v={}&intent={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            category_query,
            VERSION, 
            "browse",
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        #print (url)
        # make the GET request
        results = requests.get(url).json()["response"]
        #print (results)
        if len (results["venues"] )== 0:
            continue
        for v in results["venues"]:         
            cat = getJsonField(v, 'categories')
            if (cat !='None'):
               cat = getJsonField(cat[0], 'name')
               #print(cat)
            else:
               cat='None'
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['id'],
                v['name'], 
                v['location']['lat'], 
                v['location']['lng'],  
                cat)])
        

        cat_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        cat_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude',
                  'Id',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(cat_venues)

In [50]:
category_query = '4bf58dd8d48988d1d3941735'  # Vegatarian 
all_veg_venues = getVenuesOfCategory(df_nyc['Neighborhood'],df_nyc['Latitude'],df_nyc['Longitude'],category_query, radius=500 )

In [51]:
project.save_data(data=all_veg_venues.to_csv(index=False),file_name='Raw_NYC_BLY_Veg_Venues.csv',overwrite=True)

{'file_name': 'Raw_NYC_BLY_Veg_Venues.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'battleneighborhood-donotdelete-pr-daicfvssjqzcx5',
 'asset_id': '49daaab3-462a-4037-ab7d-ed760bba1f7b'}

## (4) Saving Details of Veg Restaruants to file
###  Raw_Veg_Details.csv
Since getting the details is a Premium call and my account is limited to 50 calls per day, this program is used to accumulate the details everyday till we get all the details

In [1]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,52411ca98bbd75ed0f690c1f,Kingsbridge-Riverdale Farmers' Market,40.879394,-73.907125,Vegetarian / Vegan Restaurant
1,Greenpoint,40.730201,-73.954241,4b9709fcf964a520c4f434e3,Paulie Gee’s,40.729801,-73.958520,Pizza Place
2,Greenpoint,40.730201,-73.954241,53dd1ead498e9ef3f539fb4e,Jungle Cafe,40.730201,-73.954761,Vegetarian / Vegan Restaurant
3,Greenpoint,40.730201,-73.954241,4ff37cb9e4b0b8fda883be7f,Adelina's,40.730301,-73.953352,Italian Restaurant
4,Greenpoint,40.730201,-73.954241,59c68c0c4382ab16da7cf734,Brooklyn Whiskers,40.733314,-73.954791,Bakery


In [3]:
df_veg_id = pd.DataFrame( df_veg['Id'].unique())
df_veg_id.columns=['Id']

In [6]:
def getDetails(listID):
    

   detail_list = []
   for  id in listID:
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION )
        print (id)
              
        # make the GET request
        jresults=requests.get(url).json()
  
        results = jresults["response"]
     
        if not results:
            print ("No results for " + id)
            continue
        venue = results['venue']
        price = getJsonField(venue, 'price')
        if (price != 'None'):
            price = getJsonField(price, 'message')
        else:
            price='None'
        

        detail_list.append([
            id,
            venue['name'],
            price,
            getJsonField(venue,'rating')])
    

   
   vegdetails = pd.DataFrame(detail_list)
   vegdetails.columns = [
                  'Id',
                  'Name',
                  'Price',
                  'Rating' 
                  ]
   return vegdetails

In [14]:
# We call this each day ( getting 50) till we filled all the details for 200+ venues
vegi_venues_details = getDetails(df_veg_id['Id'][0:1])

52411ca98bbd75ed0f690c1f


In [12]:
## After getting all the venues, I saved the info in file Raw_Veg_Det.csv